# Count Words 개요
- 통계적 언어 모델 (SLM : Statistical Language Model)
    * 확률 분포를 기반으로 언어 이해 (ex 빈도 수 등)
    * 모델 예시 : Bow, DTM, TDM, TF-IDF
- 신경망 언어 모델 (NNLM : Neural Network Language Model)
---

## (BoW) Bag of Words
* 문서 내의 단어들의 순서나 문맥을 고려하지 않음
* 단어의 출현 빈도에만 집중
* 문서 분류, 정보 검색 등 다양한 작업(순서와 문맥을 고려하지는 않는 작업)에서 사용
* 단일 문서에 대한 처리 방법

## DTM(Document-Term Matrix)
* 텍스트 문서를 단어의 출현 빈도를 기반으로 행렬로 표현
* 각 행은 문서, 각 열은 단어
* 문서간 유사성 분석, 주제 모델링, 정보 검색 등의 작업에서 사용

## TDM(Term-Document Matrix)
* 텍스트 문서를 단어의 출현 빈도를 기반으로 행렬로 표현
* 각 행은 단어, 각 열은 문서


## TF-IDF
---

# (BoW) Bag of Words
---
* 문서 내의 단어들의 순서나 문맥을 고려하지 않음
* 단어의 출현 빈도에만 집중
* 문서 분류, 정보 검색 등 다양한 작업(순서와 문맥을 고려하지는 않는 작업)에서 사용
* 단일 문서에 대한 처리 방법

In [2]:
from konlpy.tag import Okt
okt = Okt()

def build_bag_of_words(document):
    # 토큰화 처리
    document = document.replace('.','')                 # 구두점 제거
    tokenized_document = okt.morphs(document)           # 토큰화
    
    # BoW와 단어집합 만들기 - 객체 선언
    bow = []                                            # bow
    word_to_index = {}                                  # 단어 집합
    
    # Bow와 단어집합 만들기
    for word in tokenized_document:                     # 토큰화된 단어 각각 뽑아내기
        if word not in word_to_index.keys():            # 단어 집합의 키값에에 토큰화된 단어가 없는 경우
            word_to_index[word] = len(word_to_index)        # 단어 집합에 키값 추가, 단어 집합의 현재 길이만큼(=추가된 순서, 0부터 시작)을 밸류값으로 추가
            bow.insert(len(word_to_index) - 1, 1)           # bow의 (단어 집합의 현재 길이보다 1 작은 위치 -> 인덱스는 0부터 시작하고 방금 뭔가 추가되어 길이는 1부터 시작하기 때문에)를 인덱스로 하여 1 추가
        else:                                           # 단어 집합의 키값에 토큰화된 단어가 이미 있음
            index = word_to_index.get(word)                 # 단어 집합의 word를 키값으로하는 밸류를 가져옴(추가된 순서)
            bow[index] += 1                                 # bow에 (단어 집합에 추가된 순서)를 인덱스로 해서 1 추가
    return word_to_index, bow

# list.insert(i, x) 형태로 사용한다. list의 원하는 위치 i 앞에 추가할 값 x를 삽입

In [3]:
doc1 = '정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.'
vocab, bow = build_bag_of_words(doc1)

print('vocabulary: ', vocab)
print('bag of words vector: ', bow)

vocabulary:  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector:  [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [4]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = build_bag_of_words(doc2)
print('vocabulary: ', vocab)
print('bag of words vector: ', bow)

vocabulary:  {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector:  [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [5]:
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)

print('vocabulary: ', vocab)
print('bag of words vector: ', bow)

vocabulary:  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector:  [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


 #### CountVectorize로 BoW 만들기

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you knoe I want your love. because I love you.']
vector = CountVectorizer()

print('bag of words vector: ', vector.fit_transform(corpus).toarray())
print('vocabulary: ',vector.vocabulary_)

bag of words vector:  [[1 1 2 1 2 1]]
vocabulary:  {'you': 4, 'knoe': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


#### 불용어를 제거한 BoW 만들기

##### 사용자 정의 불용어

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text = ["Family is not an important thing. It's everything."]           # 대상 corpus
vect = CountVectorizer(stop_words=['the','a','an','is','not'])          # CountVectorizer 객체 선언하면서 stop_words 속성에 불용어 정의

print('bag of words vector: ', vect.fit_transform(text).toarray())
print('vocabulary: ',vect.vocabulary_)

bag of words vector:  [[1 1 1 1 1]]
vocabulary:  {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


##### CounterVectorizer 내장 불용어

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text = ["Family is not an important thing. It's everything."]           # 대상 corpus
vect = CountVectorizer(stop_words='english')                            # stopwords 속성에 필터링할 불용어 매칭

print('bag of words vector: ', vect.fit_transform(text).toarray())
print('vocabulary: ',vect.vocabulary_)

bag of words vector:  [[1 1 1]]
vocabulary:  {'family': 0, 'important': 1, 'thing': 2}


##### nltk 내장 불용어

In [14]:
stop_words = stopwords.words('english')                                 # nltk stopwords에서 .words 명령어로 불용어 셋 가져오기

text = ["Family is not an important thing. It's everything."]           # 대상 corpus
vect = CountVectorizer(stop_words=stop_words)                           # stopwords 속성에 필터링할 불용어 매칭

print('bag of words vector: ', vect.fit_transform(text).toarray())
print('vocabulary: ',vect.vocabulary_)

bag of words vector:  [[1 1 1 1]]
vocabulary:  {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}
